
<h1><center> ptfesc.ipynb </center></h1>

<br>
<br>
<br>
<br>
<br>

Last Modified: By Daniel Baur on 18th January 2021

This jupyter notebook is used for the analysis of the PTFE screening campaign. For a comprehensive elaboration of the study, have a look at the corresponding [wiki note](https://wiki.uni-freiburg.de/app/doku.php?id=start:ultimate:monxe:monxe_studies_ptfe_screening_campaign).

<br>
<br>

### Table of Contents


0. **[Stuff](#0.-Stuff)**<br>
    0.1 [Documentation: Using the GeMSE Analysis Infrastructure on bcfr102](#0.3-Documentation:-Using-the-GeMSE-Analysis-Infrastructure-on-bcfr102)<br>
    0.2 [Documentation: Setting Up the GeMSE Analysis Infrastructure on my Private Laptop](#0.4-Documentation:-Setting-Up-the-GeMSE-Analysis-Infrastructure-on-my-Private-Laptop)<br>
    0.3 [Documentation: Creating a Docker Image for the GeMSE Analysis Infrstructure](#0.5-Documentation:-Creating-a-Docker-Image-for-the-GeMSE-Analysis-Infrstructure)<br>
    0.4 [Imports](#0.1-Imports)<br>
    0.5 [Definitions](#0.2-Definitions)<br>


1. **[Analysis: GeMSE](#1.-Analysis:-GeMSE)**<br>
    1.1 [cuboid_01_gdflw](#1.1-cuboid_01_gdflw)<br>
    1.2 [cuboid_02_ekcln](#1.2-cuboid_02_ekcln)<br>
    1.3 [cuboid_03_ekreg](#1.3-cuboid_03_ekreg)<br>
    1.4 [cuboid_04_lanza](#1.4-cuboid_04_lanza)<br>
    1.5 [cuboid_05_grfln](#1.5-cuboid_05_grfln)<br>
    1.6 [powder_06_cnb16](#1.6-powder_06_cnb16)<br>
    1.7 [powder_07_cnb11](#1.7-powder_07_cnb11)<br>


<br>
<br>
<br>
<br>
<br>

# 0. Stuff

<br>
<br>


<br>
<br>

## 0.1 Documentation: Using the GeMSE Analysis Infrastructure on bcfr102

<br>

The GeMSE measurement data is analyzed utilizing the GeMSE analysis framework installed on ``bcfr102``, an old server that was brought from Bern to Freiburg. You can access the machine via ``ssh`` - but only from ``pcfr31``. 

(In order to do so you first need to generate an ssh key pair. Send the public one to Diego; he needs to register it on the GeMSE machine. Also implement the other one into your ssh configuration file on `pcfr31`. Also see [here](https://askubuntu.com/questions/311558/ssh-permission-denied-publickey) (I had to add `IdentitiesOnly yes`).)

<br>

### Data Submission

1. Copy the `data` directory onto `pcfr31`:<br>
`$ scp -r ./data dbaur@pcfr31.physik.uni-freiburg.de:~/notebooks/ptfesc/data`

2. Log into `pcfr31` (PW: %b%%%1%%%!):<br>
`$ ssh -XY dbaur@pcfr31.physik.uni-freiburg.de`

3. Once logged in in `pcfr31`, copy the `data` directory further onto `bcfr102`:
`$ scp -r ~/notebooks/ptfesc/data gemse@bcfr102:~/daniel/data`<br>
(Note: For whatever reasons you first need to change to the `.ssh` directory in order to be able to access `bcfr102`. I'll edit this later on.)

4. Log into `bcfr102`:<br>
`$ ssh -Y gemse@bcfr102`

<br>

### GeMSE Analysis

The following directories within `bcfr102` are important:
- `~/daniel`: folder Diego created, containing specifically my data
- `~/daniel/20200908_ptfe_bricks_simple_analysis`: folder containing the analysis scripts Diego compiled for me
- `~/daniel/20200904__data`: folder containing the raw data
- `~/daniel/data`: folder containing the raw data (edited: 12th Dec 2020)
- `~/GeMSE_ROOT_scripts`: folder containing the root analysis scripts
- `~/GeMSE_ROOT_scripts_copy_20200904`: backup folder containing the root analysis scripts
- `~/GeMSE_analysis`: folder containing the GeMSE analysis script
- `~/GeMSE_analysis_copy_20200908`: backup folder containing the GeMSE analysis script
The steps below summarize how to run the GeMSE analysis infrastructure.

<br>

1. **Generating a Calibrated Energy Spectrum**<br>
Check out [Thomas Kok's wiki note](https://wiki.uni-freiburg.de/app/doku.php?id=start:gemse:daq:data_workflow).
  - In order to generate a calibrated energy spectrum one first needs to convert the list file saved by the MCA into a .root file. Therefore execute the following:<br><br>
`$ /path/to/backup/scripts/folder/make_rootfile_list /path/to/mca_list_file.txt /path/to/calibration_function`<br><br>
e.g.:<br><br>
`$ ~/GeMSE_ROOT_scripts_copy_20200904/make_rootfile_list ~/daniel/20200904__data/20191010__ptfe_01_gf/2019-10-28_PTFE_Daniel_ch000_Copy20191115_rightAfterClosingGN2valve.txt /home/gemse/measurements/samples/2018-10-08_RockSample_Beda/sample_spectrum/energy_calibration/calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root`<br><br>
Hereby the file `/home/gemse/measurements/samples/2018-10-08_RockSample_Beda/sample_spectrum/energy_calibration/calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root` provides the necessary energy calibration function. If you do not reference a suchlike file the output root file will be histogrammed in adc channels.<br>

  - Next one needs to generate a spectrum out of the just generated .root file. Execute<br><br>
  `$ /path/to/backup/scripts/folder/make_spectrum_list --file /path/just/generated/root/file.root --energy`<br><br>
  e.g.:<br><br><br>
  `$ ~/GeMSE_ROOT_scripts_copy_20200904/make_spectrum_list --file ~/daniel/20200904__data/20191010__ptfe_01_gf/2019-10-28_PTFE_Daniel_ch000_Copy20191115_rightAfterClosingGN2valve.txt.root --energy`<br><br>
  Hereby the flag `--energy` designates to use the just specified energy calibration. 


2. **Running the actual Analysis**

Now is the time to finally run the analysis:<br><br>
`$ path/to/GeMSE_analysis path/to/config/file.txt`<br><br>
e.g.<br><br>
`$ ~/GeMSE_analysis_copy_20200908/GeMSE_analysis ~/daniel/20200908_ptfe_bricks_simple_analysis/results/kljhsdfkl.txt`


<br>

<br>

## 0.2 Documentation: Setting Up the GeMSE Analysis Infrastructure on my Private Laptop

<br>

This section is meant to document how I set up the GeMSE analysis infrastructure on my private laptop (in order to then maybe develop a docker solution at some point in the future). See also [Thomas Kok's documentation in the wiki](https://wiki.uni-freiburg.de/app/doku.php?id=start:gemse:daq:data_setup). The local installation of the GeMSE analysis infrastructure is located in `/home/daniel/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/`.

<br>

### Installing ROOT

I basically followed the [official instructions](https://root.cern/install/):
- Within the `20210105__local_gemse_analysis_framework` project directory I created the `ROOT` install folder `root`.
- I installed/updated all of the [required dependencies](https://root.cern/install/dependencies/) via: `$ sudo apt-get install <package>`.
- I downloaded and and unpacked the [precompiled `ROOT` binaries for my Ubuntu 18.04 installation](https://root.cern/install/all_releases/) via: `wget https://root.cern/download/root_v6.22.02.Linux-ubuntu18-x86_64-gcc7.5.tar.gz`.
- I edited my `.bashrc` to automatically source the `ROOT` installation by adding the following line:<br>
`source /home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/root/root_v6.22.06.Linux-ubuntu18-x86_64-gcc7.5/root/bin/thisroot.sh`

<br>

### Installing BAT

- Within the `20210105__local_gemse_analysis_framework` project directory I created the `BAT` containing and install folders `bat_0_9_4_1` and `bat_0_9_4_1_install`.
- I downloaded and extracted the [`BATv0.9.4.1` git repository](https://github.com/bat/bat/releases/tag/v0.9.4.1) into the directory `bat_0_9_4_1`.
- From within the `BAT-0.9.4.1` directory I configured the following installation via: `$ ./configure --prefix=/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/bat_0_9_4_1_install` (by adding the prefix flag you can determine the install directory).
- Also within the `BAT-0.9.4.1` directory I could then run `$ make` and `$ make install`.
- Once the installation is done I made sure to automatically source the `BAT` installation by adding the following lines to my `.bashrc`:<br>
`BATPREFIX="/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/bat_0_9_4_1_install"`<br>
`export PATH="$BATPREFIX/bin:$PATH"`<br>
`export LD_LIBRARY_PATH="$BATPREFIX/lib:$LD_LIBRARY_PATH"`<br>
`export CPATH="$BATPREFIX/include:$CPATH"`<br>
`export PKG_CONFIG_PATH="$BATPREFIX/lib/pkgconfig:$PKG_CONFIG_PATH"`<br>

<br>

### Compiling Moritz' Repositories: GeMSE_ROOT_scripts and GeMSE_analysis

- Within the `20210105__local_gemse_analysis_framework` project directory I created the containing folders `gemse_root_scripts` and `gemse_analysis`.
- Therein I downloaded and unpacked the git repositories [`GeMSE_ROOT_scripts`](https://github.com/vonsivers/GeMSE_ROOT_scripts) and [`GeMSE_analysis`](https://github.com/vonsivers/GeMSE_analysis) respectively:<br>
`git clone https://github.com/vonsivers/GeMSE_ROOT_scripts.git`<br>
`git clone https://github.com/vonsivers/GeMSE_analysis`<br>
- Unfortunately I had to manually replace `ifstream` with `std::ifstream` within all `.cxx` files for the `GeMSE_ROOT_scripts` repository (probably due to updates of `ROOT` or some `C++` compiler; thank you Sebastian).
- In order to properly compile the executables I executed the following within the respective directories:<br>
`$ make clean`<br>
`$ make`<br>
`$ make install`

<br>

### Comments

Setting up the GeMSE analysis infrastructure (even just partially) on my laptop took more time than I'd like to admit. Below you find some of the thoughts I thought were worth writing down.
- `bcfr102` runs `ROOTv5.34/34`.
- `bcfr102` runs `BATv0.9.4.1`.
- Moritz' `GeMSE_analysis` script does not compile with the most recent (4th January 2021) `BAT` release (!).
- For documentation of Moritz' scripts look at the readme files within his repositories.
- loose ends:
  - energy calibration: I use some already-existing file.
  - energy resolution: I use some already-existing file.
  - efficiency simulation: I use some already-existing file.
  - analysis settings: I use some already-existing settings.

<br>


<br>

## 0.3 Documentation: Creating a Docker Image for the GeMSE Analysis Infrstructure

<br>

This section is meant to provide documentation (mainly for myself) on how I set up the GeMSE analysis Docker image.

<br>

#### Resources

- [excellent Docker tutorial on YouTube](https://www.youtube.com/watch?v=3c-iBn73dDE)
- contacts: Diego Ramirez, Frank Sauerburger (AG Jakobs), Matthias Gorzellik
- 

<br>

#### Useful Docker Commands

- `$ docker run --rm -it <pathstring_image>`:<br>
  The (probably most important) command `docker run` is used to eventually pull the specified image and run a new container from the specified image.
  - `--rm`:
  - `-it`: running the container interactively will open a terminal within it
  - `-d`: running the container in detached mode will run the containered process in the background
  - `-p:<container_port>:<host_port>`: attache the container to the specified ports
  - `-e<ev_name>=<ev_value>`: specify an environmental variable
  - `--name <custom_container_name>`: specify the container name
  - `--net <network_name>`: specify the container network within which the container is run


- `$ docker images`:<br>
  The command `docker images` displays the locally available images.


- `$ docker ps`:<br>
  The command `docker ps` lists all currently running containers along with relevant information.
  - `-a`: also list stopped containers.


- `$ docker stop <container_id>`:<br>
  The command `docker stop` stops the specified container currently running.


- `$ docker start <container_id>`:<br>
  The command `docker start` starts a stopped container.


- `$ docker exec -it <container_id> /bin/bash`:<br>
  The command `docker exec` is opening up a terminal within the specified container.


- `$ docker rm <container_id>`:<br>
  The command `docker rm` is deleting a container.


- `$ docker rmi <image_id>`:<br>
  The command `docker rmi` is deleting an image.


- `$ docker-compose -f <pathstring_docker_compose_yaml_file> up`:<br>
  The command `docker-compose` will set up the container network specified within the specified `.yaml` file.
  - `-f <pathstring_docker_compse_yaml_file>`: specify the utilized compose file
  - note: depending on whether you specify `up` or `down` the command `docker-compose` will either boot or shut down the containers specified within the indicated compose file.

<br>

#### Installing Docker

In order to install `Docker` I just followed the instruction for my `Ubuntu:18.04` system in the [official documentation](https://docs.docker.com/engine/install/ubuntu/).

<br>

#### Building the Docker Image and Running the Container

- **Setting Up the `Dockerfile`:**<br>
  The so-called `Dockerfile` is basically the blueprint for the image you are about to create. Name it `Dockerfile` is mandatory. The syntax is relatively easy to understand:
  - `FROM`: indicating the image your image is based upon.
  - `SHELL ["/bin/bash", "-c"]`: indicating that the following `RUN` commands are supposed to utilizing the standard Ubuntu shell.
  - `RUN`: Listed here are commands that are executed from WITHIN the image


- **Building the Image:**<br>
  To build the `gemse_analysis:1.0` image execute the following from within the  directory that contains the `Dockerfile` (i.e., `20210114__docker_image_for_gemse_analysis`):<br>
  `$ docker build -t gemse_analysis:1.0 .` (don't forget the `.` indicating the `Dockerfile` saved within this directory)


- **Running the Container:**<br>
  Run the `gemseana` docker container with an interactive terminal (`-it`) based upon the just created `gemse_analysis:1.0` image:<br>
  `$ sudo docker run --name gemseana -it gemse_analysis:1.0`


- **Jupyter Notebook:**<br>
  Execute from within the container:<br>
  `$ jupyter notebook --port=8880 --no-browser --ip=0.0.0.0 --allow-root`<br>
  On your local machine open any browser and access the followiwng URL<br>
  `http://127.0.0.1:8880/?token=<token>`<br>
  with `<token>` being the forwarded token from the above `jupyter notebook` call.<br>
  This now enables you work with a `jupyter notebook` environment with all prerequisites installed.


- **Notes:**<br>
  - **occupied port**: In the example above I used the (arbitrarily chosen) port `8880`. Should that port already be occupied, just choose another one (e.g., `8888`, `8889`, etc...).
  - **workflow when modifying the Docker image:**<br>
  Within the just opened terminal you can now see whether the system works as inteded. If you'd like to make adjustments to the image you first have to remove the container `gemseana` and also the image `gemse_analysis:1.0`:<br>
  `$ sudo docker rm gemseana; sudo docker rmi gemse_analysis:1.0`<br>
  Once you are happy with your modifications to the image just rebuild the image and run the container:<br>
  `$ sudo docker build -t gemse_analysis:1.0 .; sudo docker run -p 8880:8880 --name gemseana -v /home/daniel/Desktop/dtdv:/home/gemse_analysis_infrastructure/ex -it gemse_analysis:1.0`
  


<br>



<br>

## 0.3.2 Documentation: Using the GeMSE Analysis Infrstructure Docker Image

<br>
<br>

If you have been referred to this repository because you'd like to perform some GeMSE analysis on your local machine then this section is made specifically for you! Just follow the steps below that I exemplarily adapted to fit my installation of `Ubuntu:18.04`.

<br>

#### Prerequisites: Installing Docker and Pulling the GeMSE Analysis Image

- In order to install `Docker` follow the instructions for your local system in the [official documentation](https://docs.docker.com/engine/install/ubuntu/).

<br>

#### Pulling the Docker Image

Obtain the GeMSE analysis Docker image by executing:<br>
`$ docker pull gemse_analysis:latest`

<br>

#### Running the Container
Run the `gemseana` container with an interactive terminal (`-it`) based upon the just downloaded `gemse_analysis:1.0` image:<br>
  `$ sudo docker build -t gemse_analysis:1.0 .; sudo docker run -p 8880:8880 --name gemseana -v /home/daniel/Desktop/dtdv:/home/gemse_analysis_infrastructure/ex -it gemse_analysis:1.0`

<br>

#### Jupyter Notebook

Execute from within the container:<br>
`$ jupyter notebook --port=8880 --no-browser --ip=0.0.0.0 --allow-root`<br>
On your local machine open any browser and access the followiwng URL<br>
`http://127.0.0.1:8880/?token=<token>`<br>
with `<token>` being the forwarded token from the above `jupyter notebook` call.<br>
This now enables you work with a `jupyter notebook` environment with all prerequisites installed.

<br>

#### Notes ####

- **occupied port**: In the example above I used the (arbitrarily chosen) port `8880`. Should that port already be occupied, just choose another one (e.g., `8888`, `8889`, etc...).

- **workflow when modifying the Docker image:**<br>
Within the just opened terminal you can now see whether the system works as inteded. If you'd like to make adjustments to the image you first have to remove the container `gemseana` and also the image `gemse_analysis:1.0`:<br>
  `$ sudo docker rm gemseana; sudo docker rmi gemse_analysis:1.0`<br>
  Once you are happy with your modifications to the image just rebuild the image and run the container:<br>
  `$ sudo docker build -t gemse_analysis:1.0 .; sudo docker run -p 8880:8880 --name gemseana -v /home/daniel/Desktop/dtdv:/home/gemse_analysis_infrastructure/ex -it gemse_analysis:1.0`
  


<br>


<br>

## 0.4 Imports

<br>


In [1]:

# including the monxe_software libraries
import getpass
import sys
abspath_miscellaneous_figures = "/home/daniel/Desktop/arbeitsstuff/20180705__monxe/monxe_software/miscfig/"
abspath_monxeana = "/home/daniel/Desktop/arbeitsstuff/20180705__monxe/monxe_software/monxeana/"
abspath_gemseana = "/home/daniel/Desktop/arbeitsstuff/20180705__monxe/monxe_software/gemseana/"
sys.path.append(abspath_miscellaneous_figures)
sys.path.append(abspath_monxeana) 
sys.path.append(abspath_gemseana) 
import monxeana
import gemseana
import Miscellaneous_Figures as miscfig


# matplotlib, numpy, scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('file://' +abspath_miscellaneous_figures +"danielsmplstyle.mplstyle")
import numpy as np
from scipy import stats
from scipy.special import binom as binomcoeff
from scipy.optimize import curve_fit
from scipy.integrate import quad


# other
import datetime
import pprint
import itertools
import math
import os
from matplotlib.ticker import AutoMinorLocator
import subprocess
import json



<br>

## 0.5 Definitions

<br>


In [2]:

# stuff
gemse_mint = "#5a8fa3" # mint green color of the GeMSE logo


### paths: gemse analysis infrastructure
abspath_gemse_analysis_infrastructure = "/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/"
# root
abspath_root = abspath_gemse_analysis_infrastructure +"root/root_v6.22.06.Linux-ubuntu18-x86_64-gcc7.5/root/"
pathstring_thisroot = abspath_root +"bin/thisroot.sh"
# bat
abspath_bat = abspath_gemse_analysis_infrastructure + "bat/bat/"
abspath_bat_install = abspath_gemse_analysis_infrastructure + "bat/bat_install/"
# moritz' scripts
abspath_gemse_root_scripts = abspath_gemse_analysis_infrastructure + "gemse_root_scripts/GeMSE_ROOT_scripts/"
abspath_gemse_analysis = abspath_gemse_analysis_infrastructure + "gemse_analysis/GeMSE_analysis/"
abspath_gemse_analysis_files = abspath_gemse_analysis_infrastructure + "gemse_analysis_files/"
abspath_gemse_analysis_calibration = abspath_gemse_analysis_files +"calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root"
abspath_gemse_analysis_background = abspath_gemse_analysis_files +"background_combined.root"
abspath_gemse_analysis_efficiency = abspath_gemse_analysis_files +"simulation/simulated_efficiencies.root"
abspath_gemse_analysis_resolution = abspath_gemse_analysis_files +"komar_gaon_ch00_full.dat.root_spectrum_calibrated_0-1188660s.root_resolution_function.root"
abspath_gemse_analysis_isotopes = abspath_gemse_analysis_files +"isotope_parameters/"


# paths: raw data
abspath_gemse_measurements = "/media/daniel/2_intenso_256gb_stick/gemse_measurements/"


# paths: ptfesc
abspath_ptfesc = "/home/daniel/Desktop/arbeitsstuff/20180705__monxe/monxe_studies/ptfesc/"


<br>
<br>
<br>
<br>
<br>

# 1. Analysis: GeMSE

<br>
<br>

Within this section the individual samples are analyzed with respect to their and surface emanation.

<br>
<br>
<br>

## 1.1 cuboid_01_gdflw

<br>

[ELOG](http://login.physik.uni-freiburg.de:8284/GeMSE/218)

#### Definitions

In [3]:

sample_id = "cuboid_01_gdflw"
abspath_measurement_folder = abspath_gemse_measurements +"2019-10-10_cuboid_01_gdflw/"
mca_list_files = [
    "2019-10-10_PTFE_Daniel_ch000.txt", # 0.8 days, cut first 15 000 s due to exponential fall
    "2019-10-28_PTFE_Daniel_ch000.txt"] # 27.7 days, use only first 1500*1000 s (17.4 days) due to empty dewar


#### Rates

In [4]:

if [True, False][1]:

    # looping over all mca list files
    for pathstring_raw_data in [abspath_measurement_folder +i for i in mca_list_files]:

        # mca list file ---> root file
        gemseana.make_rootfile_list(
            input_pathstring_mca_list_file = pathstring_raw_data,
            input_pathstring_calibration_function = abspath_gemse_analysis_calibration)

        # root file ---> calibrated spectrum
        gemseana.make_spectrum_list(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
            input_time_window = [0,0])

        # plot rate for all adc channels
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = False,
            input_binwidth = 1800, # s
            input_pulse_height_range = [0,16383]) # adc channels

        # plot rate for energy window
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = True,
            input_binwidth = 1800,
            input_pulse_height_range = [50,2700]) # keV


In [5]:

if [True, False][1]:

    print(f"2019-10-10: {70000/(60*60*24)} days")
    print(f"2019-10-28: {(2400*1000)/(60*60*24)} days")
    print(f"2019-10-28, first half: {(1500*1000)/(60*60*24)} days")

# According to the ELOG the (first) measurement was started on 10th October 2019 (writing to: '2019-10-10_PTFE_Daniel_ch000.txt') and crashed on 18th October 2019 (the crash was noticed on 28th October 2019).
# But for whatever reason the first file only contains 68734 s (~0.8 days) worth of data. Maybe the measurement was restarted on 18th October overwriting the first file (perhaps due to not renaming the output list file; looking at the exponential decrease still visible in the corresponding rate plot it might also be the case that the DAQ crashed way earlier but for whatever reason there was still a writing process on 18th October 2019).
# Of those 0.8 days the first 15 000 s (~0.17 days) have to be discarded due to an exponential decrease visible in the rate plot.
# The second measurement was started on 28th October 2019, accumulating 2410035 s (~27.8 days) worth of data.
# Of those 27.8 days only the first 1500*1000 s (~17.4 days) can be utilized due to an increase in the rate (probably due to an empty dewar, see 'http://login.physik.uni-freiburg.de:8284/GeMSE/221')
# This leaves a total of 0.2 days +17.4 days = 17.6 days worth of data...


 #### Analysis

In [6]:

if [True, False][1]:

    ### running the GeMSE analysis framework
    gemseana.all_in_one_gemse_analysis(
        input_pathstrings_mca_list_files = [abspath_measurement_folder +i for i in mca_list_files],
        input_time_windows = [
            [[15000,68734]], # total: 68734 s
            [[0,1500*1000]]], # total: 2410035 s
        input_pathstring_gemse_analysis_configuration_file = gemseana.gen_analysis_configuration_file(
            pathstring_output = abspath_measurement_folder +"analysis_configuration_file.txt",
            sample_name = sample_id,
            abspath_isotope_parameters_folder = abspath_gemse_analysis_isotopes,
            abspath_sample_spectrum_root_file = abspath_measurement_folder +"final_calibrated_added_spectrum.root",
            abspath_background_spectrum_root_file = abspath_gemse_analysis_background,
            abspath_efficiency_root_file = abspath_gemse_analysis_efficiency,
            abspath_resolution_root_file = abspath_gemse_analysis_resolution,
            abspath_results_folder = abspath_measurement_folder,
            accuracy_of_mcmc = ["low", "medium", "high"][0],
            threshold_on_bayes_factor = 0.33,
            cl_for_activity_limit = 0.95,
            fract_uncert_efficency = 0.02,
            list_isotopes_to_analyze = ["U238", "Ra226", "Th228", "Ra228", "Co60", "K40", "Cs137", "#Mn54", "#Ti44", "#Na22", "#Al26"]),
        input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
        input_abspath_gemse_analysis = abspath_gemse_analysis,
        input_pathstring_calibration_function = abspath_gemse_analysis_calibration)
        # writing the analysis results in wiki syntax to file
    gemseana.gen_analysis_results_wiki_syntax_file(
        pathstring_gemse_analysis_summary = abspath_measurement_folder +sample_id +"_activities_summary.txt",
        pathstring_gemse_analysis_summary_wiki_syntax = abspath_measurement_folder +sample_id +"_activities_summary_wikisyntax.txt")



<br>
<br>

## 1.2 cuboid_02_ekcln

<br>


#### Definitions

In [7]:

sample_id = "cuboid_02_ekcln"
abspath_measurement_folder = abspath_gemse_measurements +"2020-07-07_cuboid_02_ekcln/"
mca_list_files = ["2020-07-07__ptfe_02__ek_clean.txt_ch000.txt"] #  s (~ days)


#### Rates

In [8]:

if [True, False][1]:

    # looping over all mca list files
    for pathstring_raw_data in [abspath_measurement_folder +i for i in mca_list_files]:

        # mca list file ---> root file
        gemseana.make_rootfile_list(
            input_pathstring_mca_list_file = pathstring_raw_data,
            input_pathstring_calibration_function = abspath_gemse_analysis_calibration)

        # root file ---> calibrated spectrum
        gemseana.make_spectrum_list(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
            input_time_window = [0,0])

        # plot rate for all adc channels
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = False,
            input_binwidth = 1800, # s
            input_pulse_height_range = [0,16383]) # adc channels

        # plot rate for energy window
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = True,
            input_binwidth = 1800,
            input_pulse_height_range = [50,2700]) # keV


In [9]:
print(f"2020-07-07: {2412332/(60*60*24)}")
# There is only one mca file for this measurement (the other are relevant for the software-based veto test).
# This file contains 2412332 s (~27.92 days) worth of data acquired between 7th July 2020 and 4th August 2020.
# Of those only the interval [50*1000, 2150*1000] can be used due to an exponential fall (probably due to radon) and and an increase (due to an empty dewar) in the observed rate.

2020-07-07: 27.92050925925926


#### Analysis

In [10]:

if [True, False][1]:

    ### running the GeMSE analysis framework
    gemseana.all_in_one_gemse_analysis(
        input_pathstrings_mca_list_files = [abspath_measurement_folder +i for i in mca_list_files],
        input_time_windows = [[[50*1000, 2150*1000]]], # total: 2412332 s
        input_pathstring_gemse_analysis_configuration_file = gemseana.gen_analysis_configuration_file(
            pathstring_output = abspath_measurement_folder +"analysis_configuration_file.txt",
            sample_name = sample_id,
            abspath_isotope_parameters_folder = abspath_gemse_analysis_isotopes,
            abspath_sample_spectrum_root_file = abspath_measurement_folder +"final_calibrated_added_spectrum.root",
            abspath_background_spectrum_root_file = abspath_gemse_analysis_background,
            abspath_efficiency_root_file = abspath_gemse_analysis_efficiency,
            abspath_resolution_root_file = abspath_gemse_analysis_resolution,
            abspath_results_folder = abspath_measurement_folder,
            accuracy_of_mcmc = ["low", "medium", "high"][0],
            threshold_on_bayes_factor = 0.33,
            cl_for_activity_limit = 0.95,
            fract_uncert_efficency = 0.02,
            list_isotopes_to_analyze = ["U238", "Ra226", "Th228", "Ra228", "Co60", "K40", "Cs137", "#Mn54", "#Ti44", "#Na22", "#Al26"]),
        input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
        input_abspath_gemse_analysis = abspath_gemse_analysis,
        input_pathstring_calibration_function = abspath_gemse_analysis_calibration)
        # writing the analysis results in wiki syntax to file
    gemseana.gen_analysis_results_wiki_syntax_file(
        pathstring_gemse_analysis_summary = abspath_measurement_folder +sample_id +"_activities_summary.txt",
        pathstring_gemse_analysis_summary_wiki_syntax = abspath_measurement_folder +sample_id +"_activities_summary_wikisyntax.txt")



<br>
<br>

## 1.3 cuboid_03_ekreg

<br>


#### Definitions

In [11]:

sample_id = "cuboid_03_ekreg"
abspath_measurement_folder = abspath_gemse_measurements +"2020-08-24_cuboid_03_ekreg/"
mca_list_files = ["20201105__cuboid_03_ekreg__old_veto__list_file_ch000.txt"] #  s (~ days)


#### Rates

In [12]:

if [True, False][1]:

    # looping over all mca list files
    for pathstring_raw_data in [abspath_measurement_folder +i for i in mca_list_files]:

        # mca list file ---> root file
        gemseana.make_rootfile_list(
            input_pathstring_mca_list_file = pathstring_raw_data,
            input_pathstring_calibration_function = abspath_gemse_analysis_calibration)

        # root file ---> calibrated spectrum
        gemseana.make_spectrum_list(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
            input_time_window = [0,0])

        # plot rate for all adc channels
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = False,
            input_binwidth = 1800, # s
            input_pulse_height_range = [0,16383]) # adc channels

        # plot rate for energy window
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = True,
            input_binwidth = 1800,
            input_pulse_height_range = [50,2700]) # keV

# Only one file ('20201105__cuboid_03_ekreg__old_veto__list_file_ch000.txt') worth of  s (~ days) of data.
# The others contain data corresponding to a software-based muon veto test.
# '20201105__cuboid_03_ekreg__old_veto__list_file_ch000.txt': use [0,0] (DAQ startet one day after dewar exchange, DAQ stopped before sample and dewar exchange)


#### Analysis

In [13]:

if [True, False][1]:

    ### running the GeMSE analysis framework
    gemseana.all_in_one_gemse_analysis(
        input_pathstrings_mca_list_files = [abspath_measurement_folder +i for i in mca_list_files],
        input_time_windows = [[[0, 0]]], # total: 2996298 s
        input_pathstring_gemse_analysis_configuration_file = gemseana.gen_analysis_configuration_file(
            pathstring_output = abspath_measurement_folder +"analysis_configuration_file.txt",
            sample_name = sample_id,
            abspath_isotope_parameters_folder = abspath_gemse_analysis_isotopes,
            abspath_sample_spectrum_root_file = abspath_measurement_folder +"final_calibrated_added_spectrum.root",
            abspath_background_spectrum_root_file = abspath_gemse_analysis_background,
            abspath_efficiency_root_file = abspath_gemse_analysis_efficiency,
            abspath_resolution_root_file = abspath_gemse_analysis_resolution,
            abspath_results_folder = abspath_measurement_folder,
            accuracy_of_mcmc = ["low", "medium", "high"][0],
            threshold_on_bayes_factor = 0.33,
            cl_for_activity_limit = 0.95,
            fract_uncert_efficency = 0.02,
            list_isotopes_to_analyze = ["U238", "Ra226", "Th228", "Ra228", "Co60", "K40", "Cs137", "#Mn54", "#Ti44", "#Na22", "#Al26"]),
        input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
        input_abspath_gemse_analysis = abspath_gemse_analysis,
        input_pathstring_calibration_function = abspath_gemse_analysis_calibration)
        # writing the analysis results in wiki syntax to file
    gemseana.gen_analysis_results_wiki_syntax_file(
        pathstring_gemse_analysis_summary = abspath_measurement_folder +sample_id +"_activities_summary.txt",
        pathstring_gemse_analysis_summary_wiki_syntax = abspath_measurement_folder +sample_id +"_activities_summary_wikisyntax.txt")



<br>
<br>

## 1.4 cuboid_04_lanza

<br>


#### Definitions

In [14]:

sample_id = "cuboid_04_lanza"
abspath_measurement_folder = abspath_gemse_measurements +"2020-12-10_cuboid_04_lanza/"
mca_list_files = [
    "20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt", # second file (the first one was empty due to crash)
    "20210102__cuboid_04_lanza__old_veto__list_file_ch000.txt", # third file (after second crash)
    "20210114__cuboid_04_lanza__old_veto__list_file_ch000_ch000.txt"] # fourth file (after third crash)


#### Rates

In [15]:

if [True, False][1]:

    # looping over all mca list files
    for pathstring_raw_data in [abspath_measurement_folder +i for i in mca_list_files]:

        # mca list file ---> root file
        gemseana.make_rootfile_list(
            input_pathstring_mca_list_file = pathstring_raw_data,
            input_pathstring_calibration_function = abspath_gemse_analysis_calibration)

        # root file ---> calibrated spectrum
        gemseana.make_spectrum_list(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
            input_time_window = [0,0])

        # plot rate for all adc channels
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = False,
            input_binwidth = 1800, # s
            input_pulse_height_range = [0,16383]) # adc channels

        # plot rate for energy window
        gemseana.plot_rate(
            input_pathstring_root_file = pathstring_raw_data +".root",
            input_energy_calibration = True,
            input_binwidth = 1800,
            input_pulse_height_range = [50,2700]) # keV

# 
# '20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt': use [0,0] ()
# '20210102__cuboid_04_lanza__old_veto__list_file_ch000.txt': use [0,0] ()
# '20210114__cuboid_04_lanza__old_veto__list_file_ch000_ch000.txt': use [0,0] ()


#### Analysis

In [16]:

if [True, False][0]:

    ### running the GeMSE analysis framework
    gemseana.all_in_one_gemse_analysis(
        input_pathstrings_mca_list_files = [abspath_measurement_folder +i for i in mca_list_files],
        input_time_windows = [
            [[0, 0]], # total: 1400380 s
            [[0, 0]], # total: 976128 s
            [[0, 0]]], # total: 324078 s
        input_pathstring_gemse_analysis_configuration_file = gemseana.gen_analysis_configuration_file(
            pathstring_output = abspath_measurement_folder +"analysis_configuration_file.txt",
            sample_name = sample_id,
            abspath_isotope_parameters_folder = abspath_gemse_analysis_isotopes,
            abspath_sample_spectrum_root_file = abspath_measurement_folder +"final_calibrated_added_spectrum.root",
            abspath_background_spectrum_root_file = abspath_gemse_analysis_background,
            abspath_efficiency_root_file = abspath_gemse_analysis_efficiency,
            abspath_resolution_root_file = abspath_gemse_analysis_resolution,
            abspath_results_folder = abspath_measurement_folder,
            accuracy_of_mcmc = ["low", "medium", "high"][0],
            threshold_on_bayes_factor = 0.33,
            cl_for_activity_limit = 0.95,
            fract_uncert_efficency = 0.02,
            list_isotopes_to_analyze = ["U238", "Ra226", "Th228", "Ra228", "Co60", "K40", "Cs137", "#Mn54", "#Ti44", "#Na22", "#Al26"]),
        input_abspath_gemse_root_scripts = abspath_gemse_root_scripts,
        input_abspath_gemse_analysis = abspath_gemse_analysis,
        input_pathstring_calibration_function = abspath_gemse_analysis_calibration)
        # writing the analysis results in wiki syntax to file
    gemseana.gen_analysis_results_wiki_syntax_file(
        pathstring_gemse_analysis_summary = abspath_measurement_folder +sample_id +"_activities_summary.txt",
        pathstring_gemse_analysis_summary_wiki_syntax = abspath_measurement_folder +sample_id +"_activities_summary_wikisyntax.txt")


all_in_one_gemse_analysis(): abspath_measurement_folder:
/media/daniel/2_intenso_256gb_stick/gemse_measurements/2020-12-10_cuboid_04_lanza/

#################################################################
all_in_one_gemse_analysis(): mca list file(s) ---> root file(s)
#################################################################

make_rootfile_list(): executing
/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/gemse_root_scripts/GeMSE_ROOT_scripts/make_rootfile_list /media/daniel/2_intenso_256gb_stick/gemse_measurements/2020-12-10_cuboid_04_lanza/20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt /home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/gemse_analysis_files/calibration_spectrum.txt.root_spectrum_0-2050707s.root_calibration_function.root

make_rootfile_list(): executing
/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210105__local_gemse_analysis_framework/ge

In [17]:
#if [True, False][1]:
#
#
#    samplestring = "cuboid_04_lanza"
#    measurement_folder = "/home/daniel/Desktop/arbeitsstuff/20200624__gemse/studies/20210114__ptfesc_gemse/cuboid_04_lanza/20201210__cuboid_04_lanza/"
#    filename_raw_data_2 = "20201216__cuboid_04_lanza__old_veto__list_file_ch000.txt" # second file (the first one was empty due to crash)
#    filename_raw_data_3 = "20210102__cuboid_04_lanza__old_veto__list_file_ch000.txt" # third file (after second crash)
#    filename_raw_data_4 = "20210114__cuboid_04_lanza__old_veto__list_file_ch000_ch000.txt" # fourth file (after third crash)
#    
#
#
#    for i, filename in enumerate([filename_raw_data_2, filename_raw_data_3], start=0):
#
#
#        print(f"\n\n" +70*"#" +f"\nfile: {filename}\n" +70*"#" +"\n")
#
#
#        ### retrieving the raw data
#        data_raw = monxeana.get_raw_data_from_list_file(
#            pathstring_input_data = measurement_folder +filename,
#            pathstring_output = "none",
#            flag_ctr = 10**10,
#            flag_daq = ["mc2analyzer", "compass_auto", "compass_custom"][0],
#            flag_debug = ["no_debugging", "debug", "debug_storewfms"][0])
#        monxeana.print_misc_meas_information(meas_ndarray=data_raw)
#
#
#        ### implementing cuts
#        data_raw_cut = data_raw[
#            (data_raw["pulse_height_adc"] <= monxeana.adc_channel_max)
#            & (data_raw["pulse_height_adc"] >= 0)
#            & (data_raw["flag_mca"] != "10")
#            & (data_raw["flag_mca"] != "16")
#        ]
#        
#        ### cutting high activity periods
#        if [True,False][1]:
#            if filename == filename_raw_data_2:
#                data_raw_cut = data_raw_cut[
#                    (data_raw_cut["timestamp_ps"] >= 500*1000*10**12)
#                    & (data_raw_cut["timestamp_ps"] <= 600*1000*10**12)
#                ]
#            elif filename == filename_raw_data_3:
#                data_raw_cut = data_raw_cut[
#                    (data_raw_cut["timestamp_ps"] >= 800*1000*10**12)
#                    & (data_raw_cut["timestamp_ps"] <= 900*1000*10**12)
#                ]
#            else:
#                continue
#        data_raw_cut_info = monxeana.print_misc_meas_information(meas_ndarray=data_raw_cut)


<br>
<br>

## 1.5 cuboid_05_grfln

<br>



<br>
<br>

## 1.6 powder_06_cnb16

<br>



<br>
<br>

## 1.7 powder_07_cnb11

<br>
